## Problem description

> ![traveling_salesman.png](./traveling_salesman.png)

## Definitions

Ive taken these from my math 3 notes from the last AP semester:
- **Graph** `G = (V, E)`, where `V` and `E` are sets of vertices and edges respectivly.
- An edge is a set of conections between vertices (at most of size 2). For example: `G = { { a, b, c }, { {a, b}, {b, c}, {c, a} } }` is a graph with vertices `a`, `b` and `c` and connection from `a` to `b`, `b` to `c` and `c` to `a`.  
- **Path** `P` in a graph `G(V,E)` consists of an alternating sequence of vertices and edges of the form: `P = (v0, e1, v1, e2, v2, … , eN, vN)`
- **Trail:** A path P is called a trail if all edges are distinct
- **Distance**: the length of the shortest path between vertices `u` and `v`, written `d(u,v)`

Ive also annotated the vertexes in the assignment illustration for future refrence:

![traveling_salesman_with_vertexes.png](./traveling_salesman_with_vertexes.png)

## SMT Solution explained



## Manual solution

I'll start with manually solving the problem.

In [3]:
import pandas as pd

df = pd.DataFrame({
    'A': [0, 6, 7, 0, 0, 0],
    'B': [6, 0, 1, 4, 3, 0],
    'C': [7, 1, 0, 0, 2, 1],
    'D': [0, 4, 0, 0, 5, 0],
    'E': [0, 3, 2, 5, 0, 3],
    'F': [0, 0, 1, 0, 3, 0],
})
df.set_index(pd.Index(['A', 'B', 'C', 'D', 'E', 'F']), inplace=True)
# df

### Custom brute force search

In [ ]:
def rec(
    v: str,         # current vertex
    visited=[],     # visited vertices
    cP=0,           # current path length
):
    visited.append(v)
    
    if len(visited) == len(df):
        print("Path:", " -> ".join(visited), "| Length:", cP)
        visited.pop()
        return visited, cP
    
    for neighbor, weight in zip(df.columns, df.loc[v]):
        if weight > 0 and neighbor not in visited:
            rec(neighbor, visited, cP + weight)
    
    visited.pop()

In [5]:
shortests = -1

for v in df.index:
    rec(v)

Path: A -> B -> C -> F -> E -> D | Length: 16
Path: A -> B -> D -> E -> C -> F | Length: 18
Path: A -> B -> D -> E -> F -> C | Length: 19
Path: A -> C -> B -> D -> E -> F | Length: 20
Path: A -> C -> F -> E -> B -> D | Length: 18
Path: A -> C -> F -> E -> D -> B | Length: 20
Path: B -> A -> C -> F -> E -> D | Length: 22
Path: B -> D -> E -> F -> C -> A | Length: 20
Path: C -> A -> B -> D -> E -> F | Length: 25
Path: C -> F -> E -> D -> B -> A | Length: 19
Path: D -> B -> A -> C -> E -> F | Length: 22
Path: D -> B -> A -> C -> F -> E | Length: 21
Path: D -> B -> E -> F -> C -> A | Length: 18
Path: D -> E -> B -> A -> C -> F | Length: 22
Path: D -> E -> F -> C -> A -> B | Length: 22
Path: D -> E -> F -> C -> B -> A | Length: 16
Path: E -> D -> B -> A -> C -> F | Length: 23
Path: E -> F -> C -> A -> B -> D | Length: 21
Path: F -> C -> A -> B -> D -> E | Length: 23
Path: F -> C -> A -> B -> E -> D | Length: 22
Path: F -> C -> E -> D -> B -> A | Length: 18
Path: F -> E -> C -> A -> B -> D |

### Using Floyd-Warshall algorithm implementation from scipy

In [6]:
from scipy.sparse import csr_array
from scipy.sparse.csgraph import floyd_warshall

In [7]:
graph = csr_array(df.to_numpy())
graph

<Compressed Sparse Row sparse array of dtype 'int64'
	with 18 stored elements and shape (6, 6)>

In [8]:
print(graph)

<Compressed Sparse Row sparse array of dtype 'int64'
	with 18 stored elements and shape (6, 6)>
  Coords	Values
  (0, 1)	6
  (0, 2)	7
  (1, 0)	6
  (1, 2)	1
  (1, 3)	4
  (1, 4)	3
  (2, 0)	7
  (2, 1)	1
  (2, 4)	2
  (2, 5)	1
  (3, 1)	4
  (3, 4)	5
  (4, 1)	3
  (4, 2)	2
  (4, 3)	5
  (4, 5)	3
  (5, 2)	1
  (5, 4)	3


In [9]:
dist_matrix, predecessors = floyd_warshall(
    csgraph=graph, 
    directed=False, 
    return_predecessors=True
)

In [10]:
dist_matrix

array([[ 0.,  6.,  7., 10.,  9.,  8.],
       [ 6.,  0.,  1.,  4.,  3.,  2.],
       [ 7.,  1.,  0.,  5.,  2.,  1.],
       [10.,  4.,  5.,  0.,  5.,  6.],
       [ 9.,  3.,  2.,  5.,  0.,  3.],
       [ 8.,  2.,  1.,  6.,  3.,  0.]])

In [11]:
predecessors

array([[-9999,     0,     0,     1,     1,     2],
       [    1, -9999,     1,     1,     1,     2],
       [    2,     2, -9999,     1,     2,     2],
       [    1,     3,     1, -9999,     3,     2],
       [    1,     4,     4,     4, -9999,     4],
       [    2,     2,     5,     1,     5, -9999]], dtype=int32)

In [12]:
import plotly.graph_objects as go
import igraph as ig

# Create the graph
g = ig.Graph.Weighted_Adjacency(df.to_numpy().tolist(), mode="undirected")

# Get node positions using a layout
layout = g.layout("kamada_kawai")
positions = {idx: pos for idx, pos in enumerate(layout)}


# Extract edges and weights
edges = g.get_edgelist()
weights = g.es["weight"]

# Function to reconstruct shortest path tree
def shortest_path_tree(predecessors):
    tree_edges = set()
    n = len(df)
    for v in range(n):
        for u in range(n):
            if u != v and predecessors[u, v] != -9999:
                parent = predecessors[u, v]
                while parent != u:
                    tree_edges.add((min(parent, v), max(parent, v)))
                    v, parent = parent, predecessors[u, parent]
    return tree_edges

# Compute shortest path tree
shortest_tree_edges = shortest_path_tree(predecessors)

# Plot
fig = go.Figure()

# Add edges
for (src, dst), weight in zip(edges, weights):
    x0, y0 = positions[src]
    x1, y1 = positions[dst]
    color = "red" if (min(src, dst), max(src, dst)) in shortest_tree_edges else "gray"
    width = 3 if color == "red" else 1
    fig.add_trace(go.Scatter(
        x=[x0, x1, None], y=[y0, y1, None],
        line=dict(width=width, color=color),
        hoverinfo="text",
        text=f"{df.index[src]} ↔ {df.index[dst]} ({weight})",
        mode="lines"
    ))

# Add nodes
fig.add_trace(go.Scatter(
    x=[pos[0] for pos in positions.values()],
    y=[pos[1] for pos in positions.values()],
    mode="markers+text",
    marker=dict(size=10, color="blue"),
    text=[df.index[idx] for idx in positions],
    textposition="top center"
))

fig.update_layout(
    showlegend=False,
    title="Graph Visualization with Shortest Path Tree",
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

fig.show()

## Sources

https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm

https://en.wikipedia.org/wiki/Shortest-path_tree

https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csgraph.floyd_warshall.html

In [13]:
nwood = 38325
print(nwood // 64, "* 64 +", nwood % 64)

598 * 64 + 53


In [14]:
nleaves = 153040
print(nleaves // 4096, "* 4096 +", nleaves % 64)

37 * 4096 + 16


In [15]:
4096 * 11 + 12000

57056